# To do:

 - Both
     - Figure out how to signal injuries
 - Mins
     - Work on those cases where role players get an increase in playtime
     - Create foul trouble signal feature
     - Work with Szn_Wins and OppSzn_Wins

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib
import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

tgt_stat = "PRA"
print('Target Stat:', tgt_stat)

Today's date: 2026-01-05
Target Stat: PRA


In [3]:
%run ./common_utils.ipynb

# ML Functions

In [4]:
def feature_importance(model, all_features):
    # Get gain importance
    importance = model.get_score(importance_type='gain')
    
    # Fill in 0 for missing features
    df_importance = pd.DataFrame({
        'feature': all_features,
        'importance': [importance.get(f, 0) for f in all_features]
    }).sort_values(by='importance', ascending=False).reset_index(drop=True)
    
    df_importance['pct'] = df_importance.importance.cumsum() / df_importance.importance.sum()
    if df_importance.shape[0] >= 50:
        with pd.option_context('display.max_rows', None):
            display(df_importance)
    else:
        display(df_importance)
    
    xgb.plot_importance(model)
    plt.show()

In [5]:
def create_baseline_model(df, pred_col, DFS):
    
    train_df, val_df, test_df = DFS

    if pred_col == 'MP':
        print('Minutes Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            'starter', 'bench', 'reserve'
        ]
    else:
        print(f'{pred_col} Stats Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            f'{pred_col}_last_3_avg', f'{pred_col}_last_5_avg', f'{pred_col}_last_10_avg',
            f'Def_{pred_col}', f'Def_L5_{pred_col}'
        ]
    
    print('Train:', len(train_df), '/ Validation:', len(val_df), '/ Test:', len(test_df))
    
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert to DMatrix (XGBoost internal format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42
    }

    # Train using native XGBoost API with early stopping
    evals = [(dtrain, "train"), (dval, "val")]
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict on test set
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R²:", r2)
    
    return bst

In [6]:
def hyperparam_tuning(DFS, pred_col, n_iter=20, early_stopping_rounds=50):
    """
    Hyperparameter tuning using native XGBoost API and DMatrix,
    with early stopping support (compatible with XGBoost 3.1.2)
    """

    train_df, val_df, test_df = DFS
    feature_cols = [col for col in train_df.columns if col not in ['Date', pred_col]]
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert datasets to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    # Hyperparameter search space
    param_dist = {
        "n_estimators": randint(300, 1500),
        "learning_rate": uniform(0.01, 0.05),
        "max_depth": randint(3, 6),
        "min_child_weight": randint(1, 8),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.7, 0.3),
        "gamma": uniform(0, 2),
        "reg_lambda": uniform(0, 5),
        "reg_alpha": uniform(0, 2)
    }

    # Sample n_iter random parameter combinations
    param_list = []
    for _ in range(n_iter):
        sample = {k: (v.rvs() if hasattr(v, "rvs") else v) for k, v in param_dist.items()}
        sample['n_estimators'] = int(sample['n_estimators'])
        sample['max_depth'] = int(sample['max_depth'])
        sample['min_child_weight'] = int(sample['min_child_weight'])
        param_list.append(sample)

    best_mae = float('inf')
    best_params = None
    best_bst = None

    # Manual hyperparameter search
    for i, params in enumerate(param_list):
        print(f"\nTrial {i+1}/{n_iter}: {params}")
        num_boost_round = params.pop('n_estimators')
        params.update({
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "device": "cuda",
            "seed": 42
        })
        evals = [(dtrain, 'train'), (dval, 'val')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )
        # Predict on validation set to compute MAE
        val_preds = bst.predict(dval, iteration_range=(0, bst.best_iteration))
        mae = mean_absolute_error(y_val, val_preds)
        print(f"Validation MAE: {mae:.4f}")
        if mae < best_mae:
            best_mae = mae
            best_params = params.copy()
            best_bst = bst

    print("\nBest validation MAE:", best_mae)
    print("Best parameters:", best_params)

    # Predict on test set using best model
    preds = best_bst.predict(dtest, iteration_range=(0, best_bst.best_iteration))
    test_df[pred_col] = y_test
    test_df[f'{pred_col}_preds'] = preds
    test_df['Team'] = team_encoder.inverse_transform(test_df["Team"])
    test_df['Opp'] = team_encoder.inverse_transform(test_df["Opp"])
    test_df['Player'] = player_encoder.inverse_transform(test_df["Player"])
    test_df['Pos'] = position_encoder.inverse_transform(test_df["Pos"])
    analyze_df = test_df[['Date', 'Team', 'Player', 'Pos', 'Opp', pred_col, f'{pred_col}_preds']]
    print("\nTest Metrics:")
    print("RMSE:", np.sqrt(mean_squared_error(y_test, preds)))
    print("MAE:", mean_absolute_error(y_test, preds))
    print("R²:", r2_score(y_test, preds))

    return best_bst, preds, y_test, analyze_df

### Create Base df

In [25]:
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for i in [2021, 2022, 2023, 2024, 2025]:
    df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
    df_actuals['Season'] = i
    df = pd.concat([df, df_actuals])

    df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
    df_schd['Season'] = i
    df2 = pd.concat([df2, df_schd])
    
    df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
    df_gms['Season'] = i
    df3 = pd.concat([df3, df_gms])
    
    df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
    df_inj['Season'] = i
    df4 = pd.concat([df4, df_inj])

df['Date'] = pd.to_datetime(df.Date)
df2['Date'] = pd.to_datetime(df2.Date)
df3['Date'] = pd.to_datetime(df3.Date)
df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
df4['Date'] = pd.to_datetime(df4.Date)

df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
df3['PR'] = df3.PTS + df3.REB 
df3['PA'] = df3.PTS + df3.AST
df3['RA'] = df3.REB + df3.AST
df3['PRA'] = df3.PTS + df3.REB + df3.AST
df3['STL_BLK'] = df3.STL + df3.BLK
df = df.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='left')

df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch['Team_type'] = 'Away'
df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch2['Team_type'] = 'Home'
df_mtch = pd.concat([df_mtch, df_mtch2])
df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch = df_mtch.sort_values(["Team", "Date"])
df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

team_encoder = LabelEncoder()
player_encoder = LabelEncoder()
team_type_encoder = LabelEncoder()
position_encoder = LabelEncoder()
status_encoder = LabelEncoder()

# Encode string cols
team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
df["Team"] = team_encoder.transform(df["Team"])
df["Opp"] = team_encoder.transform(df["Opp"])
df["Player_name"] = df.Player
df["Player"] = player_encoder.fit_transform(df["Player"])
df["Pos"] = position_encoder.fit_transform(df["Pos"])
df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
df["Status"] = status_encoder.fit_transform(df["Status"])
df_pred = df.copy()
df = df[(df.Active == 1) & (df.MP > 0)].sort_values(['Season', 'Date', 'Team', 'Player']).reset_index(drop=True)
print('base df created', datetime.now())

base df created 2026-01-05 22:01:19.084381


In [8]:
# df3_temp = df3.copy().drop('Season', axis=1)
# df4_temp = df4.copy().drop('Season', axis=1)
# # display(df3_temp[(df3_temp.Date == '2025-12-25') & (df3_temp.game_id == '20251225_CLE_NYK')])
# df_temp = df4_temp.merge(df3_temp, on=['Date', 'Team', 'Player'], how='outer')
# df_temp['game_id'] = np.where(df_temp.game_id.isnull(), )

# df_temp = df_temp[(df_temp.Date == '2025-12-25') & (df_temp.Team.isin(['CLE', 'NYK']))]
# display(df_temp)

# Minutes Projection Model

In [9]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    df['missed_games_after'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']
          .diff(-1)
          .abs()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    df['MP_diff_next_avg5'] = df.groupby('Player')['MP'].shift(-1) - df['MP_L5_avg']
    df['Injured'] = (
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -10) & (df.role == 1)) | 
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -5) & (df.role == 2))
                    ).astype(int)
    
    df['return_game'] = (df.groupby('Player')['Injured'].shift(1) == 1).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.return_game == 1, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2]), 'ramp_phase'] = 2
    df.loc[df.games_since_return >= 3, 'ramp_phase'] = 3
    df['starter_return'] = ((df.return_game == 1) & (df.role == 1)).astype(int)
    df['bench_return']   = ((df.return_game == 1) & (df.role == 2)).astype(int)
    
    df['MP_change_pct_L5'] = (df['MP_L1_avg'] - df['MP_L10_avg']) / df['MP_L10_avg']
    df['Early_stop'] = (
        (df['MP'] < 5) |
        ((df.role == 1) & (df.MP_change_pct_L5 <= -0.35)) |   # starters
        ((df.role == 2) & (df.MP_change_pct_L5 <= -0.45)) |   # bench
        ((df.role == 3) & (df.MP_change_pct_L5 <= -0.55))     # reserve
    ).astype(int)
    df['Early_stop'] = df.groupby('Player')['Early_stop'].shift(-1).fillna(0).astype(int)
    df['MP_increase'] = (
        ((df.role == 1) & (df.MP_change_pct_L5 >= 0.15)) |
        ((df.role == 2) & (df.MP_change_pct_L5 >= 0.10)) |
        ((df.role == 3) & (df.MP_change_pct_L5 >= 0.05))
    ).astype(int)
    df['MP_increase'] = df.groupby('Player')['MP_increase'].shift(-1).fillna(0).astype(int)
    
    for N in [1, 3, 5]:
        df[f'Early_stop_L{N}'] = df.sort_values(['Player', 'Date'])['Early_stop'].shift(1).rolling(N).sum()
        df[f'Injured_L{N}'] = df.sort_values(['Player', 'Date'])['Injured'].shift(1).rolling(N).sum()
    
    # Location based features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)

    df = df.drop(['Season', 'Team_type', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 
                  'PrevLocation', 'Location', 'Injured', 'missed_games_after', 'MP_diff_next_avg5', 'return_game'], axis=1)    
    
    return df

In [10]:
df_mins = df.copy()
df_mins = setup_df_mins(con, df_mins)
display(df_mins)

n = len(df_mins)
train_end = int(0.8 * n)
val_end   = int(0.9 * n)
mins_train_df = df_mins.iloc[:train_end]
mins_val_df   = df_mins.iloc[train_end:val_end]
mins_test_df  = df_mins.iloc[val_end:]
mins_DFS = (mins_train_df, mins_val_df, mins_test_df)

# mins_model = create_baseline_model(df_mins, "MP", mins_DFS)
mins_model, mins_preds, y_test_mins, analyze_df_mins = hyperparam_tuning(mins_DFS, "MP", n_iter=10)
# feature_importance(mins_model, df_mins.columns.tolist())

# mins_model.save_model("../ML_models/mins_model.json")
# print('Saved minutes model!')

,Date,Team,Opp,Player,Pos,B2B,MP,Spread,Total,team_game_num,Szn_Wins,pstszn_gm,is_OT,MP_L1_avg,MP_L3_avg,MP_L5_avg,MP_L10_avg,gms_L7_days,prev_team_mins_pct,role,recent_role_L1,recent_role_L3,recent_role_L5,missed_games,game_spread_type,games_since_return,ramp_phase,starter_return,bench_return,MP_change_pct_L5,Early_stop,MP_increase,Early_stop_L1,Injured_L1,Early_stop_L3,Injured_L3,Early_stop_L5,Injured_L5,DaysLstGm,same_arena
0,2021-10-19,2,16,71,0,0,22.98,-23.0,231.0,1,0,0,0,2.33,5.410000,3.964,6.374000,0,NaN,2,3,3,3,0,1,0,0,0,0,-0.634452,0,0,0.0,0.0,2.0,0.0,4.0,0.0,0,0
1,2021-10-19,2,16,96,3,0,3.75,-23.0,231.0,1,0,0,0,12.55,18.450000,24.864,21.474000,0,NaN,1,2,2,2,0,1,0,0,0,0,-0.415572,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,2021-10-19,2,16,112,4,0,3.75,-23.0,231.0,1,0,0,0,34.03,23.060000,24.296,28.092000,0,NaN,2,1,2,1,0,1,0,0,0,0,0.211377,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2021-10-19,2,16,211,3,0,3.75,-23.0,231.0,1,0,0,0,12.33,11.650000,11.650,11.650000,0,NaN,2,2,2,3,0,1,0,0,0,0,0.058369,1,0,0.0,0.0,1.0,0.0,3.0,0.0,0,0
4,2021-10-19,2,16,406,2,0,30.63,-23.0,231.0,1,0,0,0,2.78,4.620000,8.382,7.942857,0,NaN,1,3,3,3,0,1,0,0,0,0,-0.650000,0,0,0.0,0.0,1.0,0.0,2.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122294,2026-01-04,29,17,568,3,1,17.85,-26.0,256.0,34,9,0,0,22.83,22.686667,21.848,24.500000,3,0.095125,2,2,2,2,0,1,5,3,0,0,-0.068163,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
122295,2026-01-04,29,17,620,4,1,7.03,-26.0,256.0,34,9,0,0,19.60,11.090000,11.012,9.572000,2,0.081667,3,2,2,2,2,1,106,3,0,0,1.047639,0,0,0.0,0.0,1.0,0.0,2.0,0.0,6,0
122296,2026-01-04,29,17,646,1,1,13.75,-26.0,256.0,34,9,0,0,16.87,19.530000,20.384,22.883000,4,0.070292,2,2,2,2,0,1,68,3,0,0,-0.262771,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
122297,2026-01-04,29,17,881,4,1,26.97,-26.0,256.0,34,9,0,0,28.00,27.163333,25.328,23.471000,4,0.116667,1,1,1,1,0,1,26,3,0,0,0.192962,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0



Trial 1/10: {'n_estimators': 497, 'learning_rate': np.float64(0.03140896253287913), 'max_depth': 5, 'min_child_weight': 5, 'subsample': np.float64(0.9503915198209091), 'colsample_bytree': np.float64(0.8408427257554735), 'gamma': np.float64(1.6290675305446352), 'reg_lambda': np.float64(4.408475631356497), 'reg_alpha': np.float64(0.8158083283040536)}
Validation MAE: 2.8900

Trial 2/10: {'n_estimators': 1410, 'learning_rate': np.float64(0.023019081731103057), 'max_depth': 4, 'min_child_weight': 5, 'subsample': np.float64(0.8344581764096114), 'colsample_bytree': np.float64(0.8225319563588487), 'gamma': np.float64(0.5230206490042528), 'reg_lambda': np.float64(4.106224110541452), 'reg_alpha': np.float64(0.05983695128445521)}
Validation MAE: 2.8970

Trial 3/10: {'n_estimators': 1488, 'learning_rate': np.float64(0.05664577353256612), 'max_depth': 5, 'min_child_weight': 3, 'subsample': np.float64(0.7088895025150357), 'colsample_bytree': np.float64(0.9816673089851813), 'gamma': np.float64(0.295

In [11]:
analyze_df_mins = mins_test_df.drop(['MP', 'MP_preds'], axis=1)\
                .merge(analyze_df_mins[['Date', 'Team', 'Player', 'MP', 'MP_preds']], on=['Date', 'Team', 'Player'])
analyze_df_mins['Diff'] = analyze_df_mins['MP'] - analyze_df_mins[f'MP_preds']
analyze_df_mins['Diff2'] = abs(analyze_df_mins['Diff'])
analyze_df_mins.sort_values('Diff2', ascending=False).drop('Diff2', axis=1).head(15)
display(analyze_df_mins[(analyze_df_mins.Player == 'VJ Edgecombe')])
# display(analyze_df_mins[(analyze_df_mins.Diff < -15)])

# plt.figure(figsize=(10,6))
# plt.hist(analyze_df_mins['Diff2'], bins=30, color='skyblue', edgecolor='black')
# plt.title('Histogram of Diff')
# plt.xlabel('Diff')
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

,Date,Team,Opp,Player,Pos,B2B,Spread,Total,team_game_num,Szn_Wins,pstszn_gm,is_OT,MP_L1_avg,MP_L3_avg,MP_L5_avg,MP_L10_avg,gms_L7_days,prev_team_mins_pct,role,recent_role_L1,recent_role_L3,recent_role_L5,missed_games,game_spread_type,games_since_return,ramp_phase,starter_return,bench_return,MP_change_pct_L5,Early_stop,MP_increase,Early_stop_L1,Injured_L1,Early_stop_L3,Injured_L3,Early_stop_L5,Injured_L5,DaysLstGm,same_arena,MP,MP_preds,Diff,Diff2
1929,2025-10-22,PHI,BOS,VJ Edgecombe,SG,0,1.0,233.0,1,1,0,0,2.22,3.140000,2.790000,2.857143,0,NaN,1,3,3,3,0,1,0,0,0,0,-0.223000,0,0,0.0,0.0,1.0,0.0,3.0,0.0,0,0,42.20,27.710606,14.489394,14.489394
2389,2025-10-25,PHI,CHO,VJ Edgecombe,SG,0,4.0,246.0,2,2,0,0,42.20,42.200000,42.200000,42.200000,1,0.175833,1,1,1,1,0,1,1,2,0,0,0.000000,0,0,0.0,0.0,1.0,0.0,2.0,0.0,3,0,39.60,34.456886,5.143114,5.143114
2762,2025-10-27,PHI,ORL,VJ Edgecombe,SG,0,12.0,260.0,3,3,0,0,39.60,40.900000,40.900000,40.900000,2,0.165000,1,1,1,1,0,2,2,2,0,0,-0.031785,0,0,0.0,0.0,0.0,0.0,1.0,0.0,2,0,39.18,36.290405,2.889595,2.889595
2898,2025-10-28,PHI,WAS,VJ Edgecombe,SG,1,5.0,273.0,4,4,0,1,39.18,40.326667,40.326667,40.326667,3,0.163250,1,1,1,1,0,1,3,3,0,0,-0.028434,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,39.98,40.940903,-0.960903,0.960903
3323,2025-10-31,PHI,BOS,VJ Edgecombe,SG,0,-1.0,217.0,5,4,0,0,39.98,39.586667,40.240000,40.240000,3,0.166583,1,1,1,1,0,1,4,3,0,0,-0.006461,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,39.73,38.082825,1.647175,1.647175
3625,2025-11-02,PHI,BRK,VJ Edgecombe,SG,0,24.0,234.0,6,5,0,0,39.73,39.630000,40.138000,40.138000,3,0.165542,1,1,1,1,0,3,5,3,0,0,-0.010165,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,32.85,33.185871,-0.335871,0.335871
3961,2025-11-04,PHI,CHI,VJ Edgecombe,SG,0,-2.0,224.0,7,5,0,0,32.85,37.520000,38.268000,38.923333,3,0.136875,1,1,1,1,0,1,6,3,0,0,-0.156033,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,37.72,37.579720,0.140280,0.140280
4159,2025-11-05,PHI,CLE,VJ Edgecombe,SG,1,-11.0,253.0,8,5,0,0,37.72,36.766667,37.892000,38.751429,3,0.157167,1,1,1,1,0,1,7,3,0,0,-0.026617,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,37.22,36.479439,0.740561,0.740561
4573,2025-11-08,PHI,TOR,VJ Edgecombe,SG,1,10.0,250.0,9,6,0,0,37.22,35.930000,37.500000,38.560000,3,0.155083,1,1,1,1,0,2,8,3,0,0,-0.034751,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,35.53,37.241535,-1.711535,1.711535
4757,2025-11-09,PHI,DET,VJ Edgecombe,SG,0,-3.0,219.0,10,6,0,0,35.53,36.823333,36.610000,38.223333,4,0.148042,1,1,1,1,0,1,9,3,0,0,-0.070463,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,31.68,37.197815,-5.517815,5.517815


In [12]:
rmse = np.sqrt(mean_squared_error(y_test_mins, mins_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_mins, mins_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', 'MP_L5_avg']][df_yesterday.MP > 0]

df_yesterday['Diff'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff'] <= rmse, 1, 0)

print("\nYesterday's Results:")
print("Total Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), '/', df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff'], axis=1)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 3.7236591201780866

Yesterday's Results:
Total Accuracy (InRMSE_Range): 0.5857142857142857
41 / 70


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
0,2026-01-04,MIL,Giannis Antetokounmpo,PF,SAC,28.480701,31.62,28.829730,1
18,2026-01-04,MIL,Bobby Portis,PF,SAC,23.641373,16.08,26.425074,0
31,2026-01-04,MIL,Myles Turner,C,SAC,27.853252,31.92,27.893523,0
40,2026-01-04,MIL,Kevin Porter Jr.,PG,SAC,36.455505,39.73,35.636871,1
55,2026-01-04,MIL,Ryan Rollins,PG,SAC,31.941858,33.87,29.359307,1
63,2026-01-04,MIL,Kyle Kuzma,PF,SAC,29.801361,17.72,29.057417,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
1,2026-01-04,CLE,Sam Merrill,SG,DET,23.550919,27.25,26.418029,1
13,2026-01-04,CLE,Donovan Mitchell,SG,DET,34.247589,33.08,32.995182,1
17,2026-01-04,CLE,Evan Mobley,PF,DET,31.198790,32.75,29.981841,1
27,2026-01-04,CLE,De'Andre Hunter,SF,DET,26.888300,24.10,26.372499,1
66,2026-01-04,CLE,Jaylon Tyson,SG,DET,27.216845,25.27,27.896196,1
70,2026-01-04,CLE,Darius Garland,PG,DET,32.487499,32.13,31.767260,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
2,2026-01-04,LAL,Luka Doncic,PG,MEM,35.530140,38.88,32.367308,1
12,2026-01-04,LAL,LeBron James,SF,MEM,33.629990,37.28,31.058966,1
34,2026-01-04,LAL,Marcus Smart,SG,MEM,29.761141,31.15,25.773956,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
3,2026-01-04,MEM,Cedric Coward,SG,LAL,27.743280,12.55,27.678212,0
9,2026-01-04,MEM,Jaren Jackson Jr.,C,LAL,33.197258,33.68,31.633083,1
53,2026-01-04,MEM,Jaylen Wells,SG,LAL,30.453861,24.50,31.675684,0


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
20,2026-01-04,DEN,Jamal Murray,PG,BRK,36.312431,39.73,35.845502,1
29,2026-01-04,DEN,Christian Braun,SG,BRK,27.538017,24.20,28.551500,1
62,2026-01-04,DEN,Aaron Gordon,PF,BRK,23.628725,21.23,29.773669,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
21,2026-01-04,ORL,Tyus Jones,PG,IND,20.738050,13.92,23.592847,0
24,2026-01-04,ORL,Desmond Bane,SG,IND,34.898941,36.55,35.254938,1
38,2026-01-04,ORL,Paolo Banchero,PF,IND,36.149956,36.15,34.159427,1
41,2026-01-04,ORL,Anthony Black,PG,IND,34.568218,35.55,34.458659,1
44,2026-01-04,ORL,Wendell Carter Jr.,C,IND,31.493427,31.52,31.640240,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
23,2026-01-04,OKC,Shai Gilgeous-Alexander,PG,PHO,33.915668,35.65,30.848370,1
25,2026-01-04,OKC,Jalen Williams,SG,PHO,28.511902,34.48,28.316282,0
26,2026-01-04,OKC,Chet Holmgren,PF,PHO,29.873045,32.60,27.023697,1
35,2026-01-04,OKC,Ajay Mitchell,SG,PHO,26.103598,27.20,24.658169,1
42,2026-01-04,OKC,Cason Wallace,SG,PHO,26.628942,17.85,25.714904,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
33,2026-01-04,MIA,Andrew Wiggins,SF,NOP,32.401409,28.35,30.546752,0
48,2026-01-04,MIA,Davion Mitchell,PG,NOP,31.737719,29.32,27.887386,1
58,2026-01-04,MIA,Bam Adebayo,C,NOP,32.584702,28.78,30.225278,0
60,2026-01-04,MIA,Norman Powell,SG,NOP,32.088047,28.78,30.881419,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_L5_avg,InRMSE_Range
57,2026-01-04,BRK,Michael Porter Jr.,SF,DEN,31.644651,32.07,32.302234,1
61,2026-01-04,BRK,Terance Mann,SG,DEN,26.127554,26.03,26.598918,1


# Main Model

In [13]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

In [24]:
df_main = df[df.Season >= 2022].copy()
df_main = setup_df_main(df_main, tgt_stat)
display(df_main)

n = len(df_main)
train_end = int(0.65 * n)
val_end   = int(0.85 * n)
main_train_df = df_main.iloc[:train_end]
main_val_df   = df_main.iloc[train_end:val_end]
main_test_df  = df_main.iloc[val_end:]
main_DFS = (main_train_df, main_val_df, main_test_df)

# stat_model = create_baseline_model(df_main, tgt_stat, main_DFS)
stat_model, stat_preds, y_test_stat, analyze_df_stat = hyperparam_tuning(main_DFS, tgt_stat, n_iter=10)
# feature_importance(stat_model, df_main.columns.tolist())

# stat_model.save_model(f"../ML_models/{tgt_stat}_model.json")
# print(f'Saved {tgt_stat} model!')

,Date,Team,Opp,Player,Pos,MP,team_game_num,PTS,Off_PTS,Off_L3_PTS,Off_L5_PTS,Off_L10_PTS,Off_PTS_Rk,Def_PTS,Def_L3_PTS,Def_L5_PTS,Def_L10_PTS,Def_PTS_Rk,Spread,Total,is_OT,three_rate_L3,ft_rate_L3,eFG_L3,TS_L3,usage_proxy_L3,three_rate_L5,ft_rate_L5,eFG_L5,TS_L5,usage_proxy_L5,three_rate_L10,ft_rate_L10,eFG_L10,TS_L10,usage_proxy_L10,three_rate_weighted,ft_rate_weighted,eFG_weighted,TS_weighted,usage_proxy_weighted,MP_lst_gm,MP_L1_avg,MP_L3_avg,MP_L5_avg,MP_L10_avg,TPM_lst_gm,TPM_L1_avg,TPM_L3_avg,TPM_L5_avg,TPM_L10_avg,FG_lst_gm,FG_L1_avg,FG_L3_avg,FG_L5_avg,FG_L10_avg,FT_lst_gm,FT_L1_avg,FT_L3_avg,FT_L5_avg,FT_L10_avg,TPA_lst_gm,TPA_L1_avg,TPA_L3_avg,TPA_L5_avg,TPA_L10_avg,FGA_lst_gm,FGA_L1_avg,FGA_L3_avg,FGA_L5_avg,FGA_L10_avg,FTA_lst_gm,FTA_L1_avg,FTA_L3_avg,FTA_L5_avg,FTA_L10_avg,role,recent_role_L1,recent_role_L3,recent_role_L5,game_spread_type
0,2022-10-18,1,22,19,0,23.10,1,6.0,6.000000,6.000000,6.0,6.0,5.0,8.000000,8.0,8.0,8.0,2.0,9.0,243.0,0,0.242063,0.814815,0.803571,0.840041,0.333084,0.309127,0.633333,0.690476,0.738062,0.381789,0.282640,0.518718,0.635430,0.692661,0.413879,0.266240,0.730761,0.752829,0.794709,0.355775,NaN,27.20,25.150000,23.776,23.561,NaN,1.0,1.000000,1.0,0.9,NaN,4.0,4.333333,4.2,4.5,NaN,4.0,4.000000,3.4,2.9,NaN,3.0,1.666667,2.4,2.3,NaN,9.0,6.666667,7.4,8.4,NaN,4.0,4.000000,3.6,3.1,1,1,1,2,2
1,2022-10-18,1,22,71,0,8.28,1,1.0,1.000000,1.000000,1.0,1.0,7.0,8.000000,8.0,8.0,8.0,2.0,9.0,243.0,0,0.000000,0.000000,0.666667,0.666667,0.302402,0.000000,0.000000,0.666667,0.642045,0.286831,0.000000,0.000000,0.600000,0.594697,0.167386,0.000000,0.000000,0.660000,0.652083,0.284229,NaN,2.33,5.410000,3.964,6.374,NaN,0.0,0.000000,0.0,0.0,NaN,1.0,1.000000,0.6,0.4,NaN,0.0,0.000000,0.2,0.1,NaN,0.0,0.000000,0.0,0.0,NaN,1.0,1.333333,0.8,0.6,NaN,0.0,0.000000,0.4,0.2,2,3,3,3,2
2,2022-10-18,1,22,231,4,24.03,1,2.0,2.000000,2.000000,2.0,2.0,6.0,2.000000,2.0,2.0,2.0,1.0,9.0,243.0,0,0.083333,0.000000,0.550000,0.550000,0.321647,0.140476,0.100000,0.572857,0.582147,0.371026,0.220238,0.111111,0.543373,0.559687,0.392727,0.114167,0.041111,0.556194,0.560613,0.343569,NaN,5.68,11.733333,13.474,12.434,NaN,0.0,0.000000,0.4,0.5,NaN,1.0,2.000000,2.6,2.4,NaN,0.0,0.000000,0.4,0.5,NaN,0.0,0.333333,0.8,1.1,NaN,2.0,3.666667,4.8,4.7,NaN,0.0,0.000000,0.6,0.6,1,3,2,2,2
3,2022-10-18,1,22,317,1,23.95,1,15.0,15.000000,15.000000,15.0,15.0,2.0,15.000000,15.0,15.0,15.0,3.0,9.0,243.0,0,0.657143,0.419048,0.404762,0.457749,0.394162,0.527619,0.284762,0.409524,0.452289,0.359923,0.574365,0.228492,0.395317,0.440909,0.378902,0.610008,0.359706,0.405246,0.454427,0.382364,NaN,15.15,14.130000,15.152,15.162,NaN,1.0,1.000000,0.6,0.8,NaN,2.0,2.000000,2.0,1.9,NaN,2.0,2.333333,1.6,1.1,NaN,2.0,2.666667,2.2,2.8,NaN,5.0,4.666667,4.8,5.2,NaN,2.0,2.666667,1.8,1.3,1,2,2,2,2
4,2022-10-18,1,22,431,3,38.57,1,35.0,35.000000,35.000000,35.0,35.0,1.0,70.000000,70.0,70.0,70.0,3.0,9.0,243.0,0,0.166667,0.166667,0.916667,0.899709,0.322357,0.166667,0.166667,0.916667,0.899709,0.322357,0.166667,0.166667,0.916667,0.899709,0.322357,0.166667,0.166667,0.916667,0.899709,0.322357,NaN,27.58,15.055000,15.055,15.055,NaN,2.0,1.000000,1.0,1.0,NaN,4.0,2.500000,2.5,2.5,NaN,1.0,0.500000,0.5,0.5,NaN,2.0,1.000000,1.0,1.0,NaN,6.0,3.500000,3.5,3.5,NaN,2.0,1.000000,1.0,1.0,1,1,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94271,2026-01-04,29,17,568,3,17.85,34,10.0,9.727273,6.333333,8.2,10.0,41.0,23.321429,21.0,19.8,20.3,4.0,-26.0,256.0,0,0.158730,0.507937,0.261905,0.366190,0.343862,0.212381,0.404762,0.352857,0.436576,0.390414,0.217024,0.322659,0.429345,0.490380,0.397839,0.180655,0.458456,0.305935,0.399725,0.363226,22.83,22.83,22.686667,21.848,24.500,0.0,0.0,0.000000,0.2,0.6,3.0,3.0,1.666667,2.6,3.6,3.0,3.0,3.000000,2.8,2.2,2.0,2


Trial 1/10: {'n_estimators': 847, 'learning_rate': np.float64(0.026575765533897698), 'max_depth': 5, 'min_child_weight': 5, 'subsample': np.float64(0.7347981488957005), 'colsample_bytree': np.float64(0.779000853605334), 'gamma': np.float64(1.258782232832797), 'reg_lambda': np.float64(1.4811204142239203), 'reg_alpha': np.float64(1.7181585950783316)}
Validation MAE: 2.3793

Trial 2/10: {'n_estimators': 545, 'learning_rate': np.float64(0.03236241785402662), 'max_depth': 3, 'min_child_weight': 3, 'subsample': np.float64(0.8760726969269985), 'colsample_bytree': np.float64(0.9263451538707894), 'gamma': np.float64(0.1181133949194868), 'reg_lambda': np.float64(1.311294044556356), 'reg_alpha': np.float64(0.3847933193080413)}
Validation MAE: 2.6871

Trial 3/10: {'n_estimators': 1160, 'learning_rate': np.float64(0.04756998823397141), 'max_depth': 3, 'min_child_weight': 7, 'subsample': np.float64(0.9606320396881536), 'colsample_bytree': np.float64(0.9768154822788149), 'gamma': np.float64(1.046038

In [15]:
analyze_df_stat = main_test_df.drop([tgt_stat, f'{tgt_stat}_preds'], axis=1)\
                .merge(analyze_df_stat[['Date', 'Team', 'Player', tgt_stat, f'{tgt_stat}_preds']], on=['Date', 'Team', 'Player'])
analyze_df_stat['Diff'] = analyze_df_stat[tgt_stat] - analyze_df_stat[f'{tgt_stat}_preds']
analyze_df_stat['Diff2'] = abs(analyze_df_stat['Diff'])
analyze_df_stat.sort_values('Diff2', ascending=False).drop('Diff2', axis=1).head(15)
# display(analyze_df_stat[(analyze_df_stat.Date == '2026-01-04') & (analyze_df_stat.Team == 'LAL')])

,Date,Team,Opp,Player,Pos,MP,team_game_num,Off_PRA,Off_L3_PRA,Off_L5_PRA,Off_L10_PRA,Off_PRA_Rk,Def_PRA,Def_L3_PRA,Def_L5_PRA,Def_L10_PRA,Def_PRA_Rk,Spread,Total,is_OT,usage_proxy_L3,usage_proxy_L5,usage_proxy_L10,usage_proxy_weighted,MP_lst_gm,MP_L1_avg,MP_L3_avg,MP_L5_avg,MP_L10_avg,PTS_lst_gm,PTS_L1_avg,PTS_L3_avg,PTS_L5_avg,PTS_L10_avg,REB_lst_gm,REB_L1_avg,REB_L3_avg,REB_L5_avg,REB_L10_avg,AST_lst_gm,AST_L1_avg,AST_L3_avg,AST_L5_avg,AST_L10_avg,TPM_lst_gm,TPM_L1_avg,TPM_L3_avg,TPM_L5_avg,TPM_L10_avg,FG_lst_gm,FG_L1_avg,FG_L3_avg,FG_L5_avg,FG_L10_avg,role,recent_role_L1,recent_role_L3,recent_role_L5,game_spread_type,PRA,PRA_preds,Diff
1085,2025-04-11,ATL,PHI,Caris LeVert,SG,30.23,81,16.710526,7.666667,9.400000,12.500000,35.0,33.205479,21.666667,21.2,28.2,15.0,14.0,234.0,0,0.473825,0.504036,0.475384,0.483044,20.08,20.08,26.063333,24.444,24.709,4.0,4.0,12.666667,13.2,11.4,3.0,3.0,4.666667,4.4,3.6,5.0,5.0,3.666667,3.8,3.0,0.0,0.0,1.666667,1.4,1.2,1.0,1.0,4.000000,4.4,4.0,2,2,1,2,2,39.0,14.588118,24.411882
10658,2025-12-07,LAL,PHI,Austin Reaves,SG,39.13,23,40.550000,41.666667,38.800000,39.300000,2.0,43.000000,31.333333,40.6,37.0,24.0,4.0,220.0,0,0.604655,0.556912,0.539471,0.583814,33.32,33.32,35.830000,37.688,37.192,36.0,36.0,32.000000,33.4,30.2,3.0,3.0,4.000000,5.0,5.9,8.0,8.0,7.000000,6.4,5.5,3.0,3.0,3.666667,4.2,3.2,9.0,9.0,9.333333,9.8,9.2,1,1,1,1,1,19.0,39.531677,-20.531677
363,2025-04-06,GSW,HOU,Stephen Curry,PG,32.67,78,34.833333,34.000000,38.800000,33.600000,12.0,37.527778,33.000000,34.0,36.9,26.0,-10.0,202.0,0,0.841530,0.729543,0.619088,0.785690,32.18,32.18,34.323333,32.534,32.271,36.0,36.0,41.666667,32.2,26.1,2.0,2.0,5.000000,4.4,3.9,5.0,5.0,6.333333,6.2,5.2,7.0,7.0,7.666667,6.0,4.7,13.0,13.0,13.000000,10.0,8.0,1,1,1,1,1,13.0,32.116364,-19.116364
4560,2025-10-27,DEN,MIN,Aaron Gordon,PF,32.82,3,31.666667,31.666667,31.666667,31.666667,11.0,45.000000,41.000000,45.0,45.0,23.0,13.0,241.0,0,0.576456,0.576456,0.576456,0.576456,25.20,25.20,31.925000,31.925,31.925,17.0,17.0,33.500000,33.5,33.5,2.0,2.0,5.000000,5.0,5.0,1.0,1.0,1.500000,1.5,1.5,1.0,1.0,5.500000,5.5,5.5,5.0,5.0,11.000000,11.0,11.0,1,1,1,1,2,15.0,33.419327,-18.419327
10180,2025-12-04,MIN,NOP,Anthony Edwards,SG,31.37,22,37.277778,38.333333,41.000000,39.100000,3.0,51.913043,55.333333,68.8,56.7,29.0,9.0,241.0,0,0.668326,0.683683,0.671204,0.673221,47.45,47.45,40.233333,40.210,38.092,44.0,44.0,38.333333,37.8,31.7,5.0,5.0,3.333333,5.0,4.8,4.0,4.0,5.000000,4.8,4.3,6.0,6.0,5.000000,5.0,3.3,16.0,16.0,13.666667,13.2,10.8,1,1,1,1,2,21.0,39.148010,-18.148010
13755,2026-01-02,NOP,POR,Zion Williamson,PF,30.70,36,31.894737,35.333333,34.400000,32.500000,8.0,32.645161,14.000000,16.2,28.2,12.0,-13.0,231.0,0,0.672293,0.700190,0.674371,0.680870,29.47,29.47,29.116667,27.248,26.358,31.0,31.0,28.333333,26.2,23.6,7.0,7.0,5.333333,5.8,6.1,3.0,3.0,1.666667,2.4,2.8,0.0,0.0,0.000000,0.0,0.0,12.0,12.0,10.333333,9.0,8.2,1,1,1,1,1,45.0,27.037363,17.962637
13840,2026-01-03,ATL,TOR,Onyeka Okongwu,C,30.30,37,27.657143,36.666667,36.000000,31.200000,13.0,31.838710,22.333333,28.2,33.5,11.0,-17.0,251.0,0,0.561790,0.539034,0.459688,0.544753,29.77,29.77,30.576667,32.578,34.189,23.0,23.0,22.000000,21.8,18.1,9.0,9.0,10.333333,10.6,9.4,2.0,2.0,4.333333,3.6,3.7,3.0,3.0,2.666667,2.8,2.3,9.0,9.0,7.666667,7.6,6.6,1,1,1,1,1,11.0,28.945959,-17.945959
7635,2025-11-16,GSW,NOP,Stephen Curry,PG,28.12,15,34.750000,42.666667,35.000000,33.400000,9.0,35.307692,31.000000,33.8,36.1,14.0,18.0,230.0,0,0.830381,0.821618,0.743282,0.819042,36.05,36.05,29.853333,30.442,30.668,49.0,49.0,35.333333,31.6,29.7,4.0,4.0,3.333333,2.8,3.8,2.0,2.0,2.333333,2.4,3.7,9.0,9.0,5.000000,4.8,4.7,16.0,16.0,11.000000,10.0,9.6,1,1,1,1,3,17.0,34.871002,-17.871002
1190,2025-04-11,GSW,POR,Jimmy Butler,SF,28.07,81,27.080000,22.000000,19.200000,24.300000,13.0,34.173913,61.333333,47.2,41.1,12.0,17.0,189.0,0,0.344341,0.341844,0.388336,0.347991,38.35,38.35,30.240000,32.184,32.357,28.0,28.0,17.000000,16.2,17.5,4.0,4.0,3.666667,3.6,4.7,7.0,7.0,4.333333,4.0

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_stat, stat_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_stat, stat_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)
df_gms = df_gms.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df_gms['STL_BLK'] = df_gms.STL + df_gms.BLK
df_gms['PR'] = df_gms.PTS + df_gms.REB 
df_gms['PA'] = df_gms.PTS + df_gms.AST
df_gms['RA'] = df_gms.REB + df_gms.AST
df_gms['PRA'] = df_gms.PTS + df_gms.REB + df_gms.AST

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', tgt_stat, 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', f'{tgt_stat}_line', f'{tgt_stat}_proj', tgt_stat]][df_yesterday.MP > 0]

df_yesterday['Diff'] = df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[f'{tgt_stat}_line']
df_yesterday['Diff2'] = abs(df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[tgt_stat])
df_yesterday['Act_Res'] = np.where(df_yesterday[tgt_stat] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['Pred_Res'] = np.where(df_yesterday[f'{tgt_stat}_proj'] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['ParlayHit'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff2'] <= rmse, 1, 0)
# df_yesterday = df_yesterday[(abs(df_yesterday.Diff) > rmse)]
# df_yesterday = df_yesterday[df_yesterday.InRMSE_Range == 0]

print("Total Accuracy (ParlayHit):", ((df_yesterday.ParlayHit == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.ParlayHit == 1).sum(), "/", df_yesterday.shape[0])

print("\nTotal Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), "/", df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff', 'Diff2', 'Act_Res', 'Pred_Res'], axis=1).sort_values(f'{tgt_stat}_line', ascending=False)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[(df_yesterday.Team == tm)]) #  & (df_yesterday.ParlayHit == 1)
else:
    display(df_yesterday)

RMSE: 3.7690030359094973
Total Accuracy (ParlayHit): 0.42857142857142855
30 / 70

Total Accuracy (InRMSE_Range): 0.24285714285714285
17 / 70


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
2,2026-01-04,LAL,Luka Doncic,PG,MEM,35.530140,38.88,51.5,43.231621,53,0,0
12,2026-01-04,LAL,LeBron James,SF,MEM,33.629990,37.28,35.5,31.214378,43,0,0
34,2026-01-04,LAL,Marcus Smart,SG,MEM,29.761141,31.15,17.5,14.948503,11,1,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
0,2026-01-04,MIL,Giannis Antetokounmpo,PF,SAC,28.480701,31.62,45.5,32.993454,49,0,0
40,2026-01-04,MIL,Kevin Porter Jr.,PG,SAC,36.455505,39.73,29.5,31.691074,39,1,0
55,2026-01-04,MIL,Ryan Rollins,PG,SAC,31.941858,33.87,25.5,26.419693,21,0,0
18,2026-01-04,MIL,Bobby Portis,PF,SAC,23.641373,16.08,19.5,23.249355,20,1,1
63,2026-01-04,MIL,Kyle Kuzma,PF,SAC,29.801361,17.72,18.5,18.056938,11,1,0
31,2026-01-04,MIL,Myles Turner,C,SAC,27.853252,31.92,17.5,20.420702,20,1,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
67,2026-01-04,DET,Cade Cunningham,PG,CLE,35.431683,37.03,44.5,44.669777,40,0,0
4,2026-01-04,DET,Isaiah Stewart,C,CLE,24.026945,31.47,21.5,14.376546,13,1,1
8,2026-01-04,DET,Ausar Thompson,SF,CLE,27.259537,21.72,21.5,16.057648,18,1,1
5,2026-01-04,DET,Marcus Sasser,PG,CLE,12.129115,3.00,17.5,11.084259,2,1,0
22,2026-01-04,DET,Duncan Robinson,SF,CLE,26.208231,29.52,16.5,13.141850,17,0,0
47,2026-01-04,DET,Jaden Ivey,SG,CLE,19.048105,19.23,15.5,14.049415,9,1,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
23,2026-01-04,OKC,Shai Gilgeous-Alexander,PG,PHO,33.915668,35.65,42.5,39.196156,31,1,0
25,2026-01-04,OKC,Jalen Williams,SG,PHO,28.511902,34.48,28.5,25.322142,34,0,0
26,2026-01-04,OKC,Chet Holmgren,PF,PHO,29.873045,32.60,27.5,30.631872,28,1,1
35,2026-01-04,OKC,Ajay Mitchell,SG,PHO,26.103598,27.20,18.5,20.979067,18,0,1
42,2026-01-04,OKC,Cason Wallace,SG,PHO,26.628942,17.85,12.5,14.354012,3,0,0


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
36,2026-01-04,NOP,Zion Williamson,PF,MIA,30.150599,26.22,34.5,36.905254,19,0,0
37,2026-01-04,NOP,Trey Murphy III,SF,MIA,29.734610,38.20,29.5,27.126348,40,0,0
16,2026-01-04,NOP,Derik Queen,C,MIA,24.684788,29.62,24.5,28.598396,23,0,0
30,2026-01-04,NOP,Jordan Poole,PG,MIA,27.910013,24.60,22.5,25.421869,17,0,0
52,2026-01-04,NOP,Jeremiah Fears,PG,MIA,27.707008,26.12,19.5,18.393875,26,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
60,2026-01-04,MIA,Norman Powell,SG,NOP,32.088047,28.78,31.5,30.832180,40,0,0
58,2026-01-04,MIA,Bam Adebayo,C,NOP,32.584702,28.78,30.5,29.682154,24,1,0
33,2026-01-04,MIA,Andrew Wiggins,SF,NOP,32.401409,28.35,25.5,28.243969,15,0,0
48,2026-01-04,MIA,Davion Mitchell,PG,NOP,31.737719,29.32,20.5,21.764332,18,0,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
9,2026-01-04,MEM,Jaren Jackson Jr.,C,LAL,33.197258,33.68,28.5,33.791691,18,0,0
3,2026-01-04,MEM,Cedric Coward,SG,LAL,27.743280,12.55,23.5,31.521452,25,1,0
53,2026-01-04,MEM,Jaylen Wells,SG,LAL,30.453861,24.50,18.5,19.582409,26,1,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
65,2026-01-04,SAC,DeMar DeRozan,PF,MIL,34.672501,28.72,27.5,27.923164,18,0,0
19,2026-01-04,SAC,Russell Westbrook,PG,MIL,28.383686,37.90,27.5,23.828552,30,0,0
7,2026-01-04,SAC,Zach LaVine,SG,MIL,28.486666,35.53,21.5,27.287941,27,1,1
51,2026-01-04,SAC,Dennis Schroder,PG,MIL,26.608156,26.12,19.5,20.707081,21,1,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PRA_line,PRA_proj,PRA,ParlayHit,InRMSE_Range
50,2026-01-04,WAS,CJ McCollum,SG,MIN,31.834545,22.17,26.5,25.292303,25,1,1
46,2026-01-04,WAS,Tre Johnson,SG,MIN,24.575035,26.97,19.5,20.977434,19,0,1
11,2026-01-04,WAS,Bilal Coulibaly,SG,MIN,28.367390,27.57,18.5,23.359003,11,0,0
54,2026-01-04,WAS,Khris Middleton,SF,MIN,22.410961,17.85,16.5,15.484090,13,1,1


### Today's predictions

In [26]:
df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)
df_lines = df_lines[~(df_lines.Team.isnull())]

# Predict Mins
df_lines["Team"] = team_encoder.transform(df_lines["Team"])
df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
df_pred_mins = setup_df_mins(con, df_pred)
df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
DM_mins = xgb.DMatrix(df_pred_mins)
df_pred['MP'] = mins_model.predict(DM_mins)

# Predict Stat
df_pred = setup_df_main(df_pred, tgt_stat)
feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
df_pred = df_pred[df_pred.Date == now][feature_cols]
DM_stats = xgb.DMatrix(df_pred)
df_pred[f"{tgt_stat}_proj"] = stat_model.predict(DM_stats)

df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
            [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
if tds_picks.shape[0] >= 50:
    print(tds_picks.shape[0], 'rows')
    for tm in tds_picks.Team.unique():
        display(tds_picks[tds_picks.Team == tm])
else:
    display(tds_picks)
tds_picks.insert(0, 'Date', pd.to_datetime(now))
# partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

108 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
46,DEN,Julian Strawther,SG,PHI,4.472094,7.899960,14.5,4.319802
45,DEN,Jalen Pickett,SG,PHI,12.576862,14.919259,11.5,5.122264
39,DEN,Bruce Brown,SG,PHI,24.790676,26.361093,11.5,5.424230
62,DEN,Peyton Watson,SF,PHI,30.443949,35.406850,19.5,17.673542


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
2,UTA,Keyonte George,PG,POR,37.022026,35.621513,25.5,33.050648
72,UTA,Lauri Markkanen,PF,POR,36.773754,34.927226,26.5,30.936752
68,UTA,Brice Sensabaugh,SF,POR,24.298767,27.923011,13.5,10.886874
99,UTA,Kyle Filipowski,C,POR,19.189964,20.507002,8.5,6.371388
98,UTA,Jusuf Nurkic,C,POR,26.978952,29.738937,11.5,13.057138
67,UTA,Ace Bailey,SF,POR,21.323715,24.335147,10.5,9.232740
69,UTA,Svi Mykhailiuk,SF,POR,15.528078,17.001706,6.5,5.243758


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
105,DET,Paul Reed,C,NYK,8.634927,10.623129,10.5,3.426623
60,DET,Ausar Thompson,SF,NYK,22.625652,25.004507,12.5,8.586424
59,DET,Duncan Robinson,SF,NYK,25.287247,27.054709,11.5,8.040268
5,DET,Cade Cunningham,PG,NYK,36.395931,34.337767,27.5,29.979879
35,DET,Jaden Ivey,SG,NYK,16.481455,18.184961,9.5,7.061247
101,DET,Isaiah Stewart,C,NYK,22.657040,26.023471,11.5,9.274531
40,DET,Javonte Green,SG,NYK,19.234924,23.234632,7.5,6.822361


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
58,CHO,Brandon Miller,SF,OKC,33.513279,32.918745,17.5,24.520069
61,CHO,Kon Knueppel,SF,OKC,31.440643,28.929011,16.5,22.375679
20,CHO,Collin Sexton,SG,OKC,20.179827,22.397908,11.5,14.068389
7,CHO,LaMelo Ball,PG,OKC,26.573227,25.611779,17.5,19.929380
86,CHO,Tidjane Salaun,PF,OKC,14.278038,19.038876,5.5,7.628437
102,CHO,Moussa Diabate,C,OKC,17.261784,26.768094,7.5,6.108494
76,CHO,Miles Bridges,PF,OKC,31.031927,30.635035,16.5,15.842466


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
91,NYK,Karl-Anthony Towns,C,DET,27.961246,24.397229,20.5,17.102083
31,NYK,Miles McBride,SG,DET,28.563335,25.977923,11.5,14.341663
4,NYK,Jalen Brunson,PG,DET,37.057949,37.794852,28.5,31.019066
107,NYK,Mitchell Robinson,C,DET,17.368511,19.036309,4.5,3.162553
78,NYK,OG Anunoby,PF,DET,36.514736,34.716349,16.5,17.244873
53,NYK,Mikal Bridges,SF,DET,37.746368,37.382793,15.5,14.787788
37,NYK,Jordan Clarkson,SG,DET,18.095024,21.749371,7.5,7.771569


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
27,BOS,Derrick White,SG,CHI,36.021992,34.330841,18.5,21.783455
100,BOS,Luka Garza,C,CHI,13.900905,20.135324,9.5,7.548356
33,BOS,Anfernee Simons,SG,CHI,23.313547,26.159134,12.5,11.146168
93,BOS,Neemias Queta,C,CHI,24.146500,24.844677,9.5,10.819527
80,BOS,Sam Hauser,PF,CHI,17.956684,19.970485,8.5,8.837373
47,BOS,Jaylen Brown,SF,CHI,32.134876,31.898406,30.5,30.297983
3,BOS,Payton Pritchard,PG,CHI,35.484032,34.237302,16.5,16.684196


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
18,POR,Shaedon Sharpe,SG,UTA,31.141403,30.768853,23.5,20.798161
75,POR,Toumani Camara,PF,UTA,35.050613,32.513621,14.5,12.075374
96,POR,Donovan Clingan,C,UTA,29.873026,27.838367,12.5,13.094577
70,POR,Kris Murray,SF,UTA,24.190361,17.647433,7.5,7.859517
50,POR,Deni Avdija,SF,UTA,37.937618,36.156377,28.5,28.268494


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
21,PHI,VJ Edgecombe,SG,DEN,36.355759,35.235813,16.5,19.130424
6,PHI,Tyrese Maxey,PG,DEN,36.919872,38.048108,26.5,28.750135
25,PHI,Quentin Grimes,SG,DEN,30.093647,33.351743,11.5,10.538461
73,PHI,Paul George,PF,DEN,33.058556,33.899341,14.5,15.283573
88,PHI,Joel Embiid,C,DEN,31.213495,31.264842,25.5,25.407394


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
16,GSW,De'Anthony Melton,PG,LAC,18.399216,24.560154,8.5,6.644701
28,GSW,Moses Moody,SG,LAC,21.158497,23.038991,9.5,7.677311
23,GSW,Brandin Podziemski,SG,LAC,25.618677,25.697875,10.5,12.106546
8,GSW,Stephen Curry,PG,LAC,34.933006,34.015447,27.5,28.347919
85,GSW,Quinten Post,PF,LAC,17.132004,17.411909,7.5,6.812972
104,GSW,Al Horford,C,LAC,14.811117,14.752698,5.5,5.729099
84,GSW,Draymond Green,PF,LAC,22.095406,24.955659,7.5,7.633038
48,GSW,Jimmy Butler,SF,LAC,33.645409,31.355917,19.5,19.378351
